In [1]:
from Architectures.GAN import GAN
from Data.load import load
from matplotlib import pyplot as plt
import numpy as np

In [2]:
train, test = load("Pre-processed", take=5)

Loading images: 100%|██████████| 5/5 [00:01<00:00,  2.67it/s]


In [3]:
gan_model = GAN(train.shape)

C:\Users\PieHu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Interpreted image shape: (80, 96, 80, 1)


Model: "GAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 80, 96, 80, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ generator (Functional)          │ (None, 80, 96, 80, 1)  │       154,705 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ discriminator (Functional)      │ (None, 2, 2, 2, 1)     │   178,861,505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 179,016,210 (682.89 MB)

 Trainable params: 154,473 (603.41 KB)

 Non-trainable params: 178,861,737 (682.30 MB)

In [4]:
gan_model.train(train[:1], epochs=100, batch_size=10)

ValueError: Dimensions must be equal, but are 10 and 2 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, discriminator_1/conv3d_11_1/add)' with input shapes: [10,1], [1,2,2,2,1].

In [ ]:
latent_space = gan_model.encode(test)
decoded_image = gan_model.decode(latent_space)

print("Original image shape:", test.shape)
print("Latent space shape:", latent_space.shape)
print("Decoded image shape:", decoded_image.shape)